In [1]:
import polars as pl
import polars.selectors as cs
from polars_ds.pipeline import Pipeline, Blueprint

In [2]:
df = pl.read_parquet("../examples/dependency.parquet")
df.head()

ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
str,str,str,str,str,str,str,str,i64,f64,str,str,str,str,str,f64,i64,i64,f64,i64,i64,i64
"""APPC90493171225""","""Female""","""23/07/79""","""15/07/16""","""C10001""","""A""","""COM0044082""","""A""",4,2000.0,"""B001""","""P""","""N""","""S122""","""G""",0.0,null,null,null,null,0,0
"""APPD40611263344""","""Male""","""07/12/86""","""04/07/16""","""C10003""","""A""","""COM0000002""","""C""",1,3500.0,"""B002""","""P""","""Y""","""S122""","""G""",0.0,20000,2,13.25,953,10,0
"""APPE70289249423""","""Male""","""10/12/82""","""19/07/16""","""C10125""","""C""","""COM0005267""","""C""",4,2250.0,"""B003""","""G""","""Y""","""S143""","""B""",0.0,45000,4,null,null,0,0
"""APPF80273865537""","""Male""","""30/01/89""","""09/07/16""","""C10477""","""C""","""COM0004143""","""A""",4,3500.0,"""B003""","""G""","""Y""","""S143""","""B""",0.0,92000,5,null,null,7,0
"""APPG60994436641""","""Male""","""19/04/85""","""20/07/16""","""C10002""","""A""","""COM0001781""","""A""",4,10000.0,"""B001""","""P""","""Y""","""S134""","""B""",2500.0,50000,2,null,null,10,0


In [3]:
# Create a blueprint first. 
# A blueprint is a plan for a pipeline. No hard work will be done until the blueprint is materialized, which
# is when the tranforms are fitted (e.g. scale learns the mean and std from base data)
# If target is specified for the blueprint, target will be excluded from all transformations that require a fit.

bp = (
    Blueprint(df, name = "example", target = "approved")
    .lowercase() # lowercase all columns
    .linear_impute(features = ["var1", "existing_emi"], target = "loan_period")
    .impute(["existing_emi"], method = "median")
    .select(cs.numeric() | cs.by_name(["gender", "employer_category1", "city_category"]))
    .append_expr(
        pl.col("existing_emi").log1p().alias("existing_emi_log1p"),
        pl.col("loan_amount").log1p().alias("loan_amount_log1p"),
        pl.col("loan_amount").sqrt().alias("loan_amount_sqrt"),
    )
    .scale( # target is numerical, but will be excluded because blueprint is initialzied with a target
        cs.numeric().exclude(["var1", "existing_emi_log1p"]), method = "standard"
    ) # Scale the columns up to this point. The columns below won't be scaled
    .append_expr(
        pl.col("employer_category1").is_null().cast(pl.UInt8).alias("gender_is_missing")
    )
    .one_hot_encode("gender", drop_first=True)
    .woe_encode("city_category", target = "approved")
    .target_encode("employer_category1", target = "approved", min_samples_leaf = 20, smoothing = 10.0)
)

print(bp)

Blueprint name: example
Blueprint current steps: 11
Features Expected: ['ID', 'Gender', 'DOB', 'Lead_Creation_Date', 'City_Code', 'City_Category', 'Employer_Code', 'Employer_Category1', 'Employer_Category2', 'Monthly_Income', 'Customer_Existing_Primary_Bank_Code', 'Primary_Bank_Type', 'Contacted', 'Source', 'Source_Category', 'Existing_EMI', 'Loan_Amount', 'Loan_Period', 'Interest_Rate', 'EMI', 'Var1', 'Approved']



In [4]:
# Materialize the blueprint
pipe:Pipeline = bp.materialize()
# Text representation of the pipeline
pipe

Naive Query Steps: 

Step 1:
col("ID").alias("id"),
col("Gender").alias("gender"),
col("DOB").alias("dob"),
col("Lead_Creation_Date").alias("lead_creation_date"),
col("City_Code").alias("city_code"),
col("City_Category").alias("city_category"),
col("Employer_Code").alias("employer_code"),
col("Employer_Category1").alias("employer_category1"),
col("Employer_Category2").alias("employer_category2"),
col("Monthly_Income").alias("monthly_income"),
col("Customer_Existing_Primary_Bank_Code").alias("customer_existing_primary_bank_code"),
col("Primary_Bank_Type").alias("primary_bank_type"),
col("Contacted").alias("contacted"),
col("Source").alias("source"),
col("Source_Category").alias("source_category"),
col("Existing_EMI").alias("existing_emi"),
col("Loan_Amount").alias("loan_amount"),
col("Loan_Period").alias("loan_period"),
col("Interest_Rate").alias("interest_rate"),
col("EMI").alias("emi"),
col("Var1").alias("var1"),
col("Approved").alias("approved")

Step 2:
.when(col("loan_period").is_n

In [5]:
df_transformed = pipe.transform(df)
df_transformed.head()

employer_category2,monthly_income,existing_emi,loan_amount,loan_period,interest_rate,emi,var1,approved,employer_category1,city_category,existing_emi_log1p,loan_amount_log1p,loan_amount_sqrt,gender_is_missing,gender_Male
f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,u8,u8
0.346575,-0.020726,-0.157655,null,-1.250145,null,null,0,0,0.010829,0.12492,0.0,null,null,0,0
-3.369204,-0.012144,-0.157655,-0.632338,-0.283462,-1.019936,-0.197259,10,0,0.021114,0.12492,0.0,-0.586105,-0.658025,0,1
0.346575,-0.019296,-0.157655,0.181273,0.68322,null,null,0,0,0.021114,-0.597055,0.0,0.537137,0.375948,0,1
0.346575,-0.012144,-0.157655,1.710861,1.166561,null,null,7,0,0.010829,-0.597055,0.0,1.527696,1.709278,0,1
0.346575,0.025049,0.935153,0.343995,-0.283462,null,null,10,0,0.010829,0.12492,7.824446,0.683076,0.543738,0,1


In [6]:
import pickle
# The pipe object can be pickled
with open("pipe.pickle", "wb") as f:
    pickle.dump(pipe, f)

In [7]:
with open("pipe.pickle", "rb") as f:
    pipe2 = pickle.load(f)

pipe2

Naive Query Steps: 

Step 1:
col("ID").alias("id"),
col("Gender").alias("gender"),
col("DOB").alias("dob"),
col("Lead_Creation_Date").alias("lead_creation_date"),
col("City_Code").alias("city_code"),
col("City_Category").alias("city_category"),
col("Employer_Code").alias("employer_code"),
col("Employer_Category1").alias("employer_category1"),
col("Employer_Category2").alias("employer_category2"),
col("Monthly_Income").alias("monthly_income"),
col("Customer_Existing_Primary_Bank_Code").alias("customer_existing_primary_bank_code"),
col("Primary_Bank_Type").alias("primary_bank_type"),
col("Contacted").alias("contacted"),
col("Source").alias("source"),
col("Source_Category").alias("source_category"),
col("Existing_EMI").alias("existing_emi"),
col("Loan_Amount").alias("loan_amount"),
col("Loan_Period").alias("loan_period"),
col("Interest_Rate").alias("interest_rate"),
col("EMI").alias("emi"),
col("Var1").alias("var1"),
col("Approved").alias("approved")

Step 2:
.when(col("loan_period").is_n

In [8]:
df_transformed_2 = pipe2.transform(df)
df_transformed_2

employer_category2,monthly_income,existing_emi,loan_amount,loan_period,interest_rate,emi,var1,approved,employer_category1,city_category,existing_emi_log1p,loan_amount_log1p,loan_amount_sqrt,gender_is_missing,gender_Male
f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,u8,u8
0.346575,-0.020726,-0.157655,null,-1.250145,null,null,0,0,0.010829,0.12492,0.0,null,null,0,0
-3.369204,-0.012144,-0.157655,-0.632338,-0.283462,-1.019936,-0.197259,10,0,0.021114,0.12492,0.0,-0.586105,-0.658025,0,1
0.346575,-0.019296,-0.157655,0.181273,0.68322,null,null,0,0,0.021114,-0.597055,0.0,0.537137,0.375948,0,1
0.346575,-0.012144,-0.157655,1.710861,1.166561,null,null,7,0,0.010829,-0.597055,0.0,1.527696,1.709278,0,1
0.346575,0.025049,0.935153,0.343995,-0.283462,null,null,10,0,0.010829,0.12492,7.824446,0.683076,0.543738,0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-3.369204,-0.004133,-0.157655,null,1.213977,null,null,10,0,0.010829,0.12492,0.0,null,null,0,0
0.346575,0.008971,0.476174,null,0.469411,null,null,7,0,0.010829,-0.597055,7.280008,null,null,0,0
0.346575,-0.023015,-0.157655,-0.50216,0.68322,2.785431,-0.210546,2,0,0.021114,-0.081991,0.0,-0.333569,-0.46065,0,0


In [9]:
from polars.testing import assert_frame_equal
# True
assert_frame_equal(df_transformed, df_transformed_2)